[View in Colaboratory](https://colab.research.google.com/github/MansiAgarwal11/Dog-Breed-Classification/blob/master/vgg16.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!ls

datalab  drive


In [0]:
import os
os.chdir("drive/DogBreedClassification/")

In [0]:
!ls

Cnn baseline.py  model1_cnn.h5		 New Folder  train
datasep.py	 model1_predictions.csv  test.ipynb  vgg16 fine tuning.py
labels.csv	 model1_vgg16.h5	 test.py     vgg16.ipynb


In [0]:
import keras

#installing vgg16
vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary()
type(vgg16_model) #not Sequential model

Using TensorFlow backend.


553467904/553467096 [==============================] - 6s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_po

keras.engine.training.Model

In [0]:
#transorming the type to Sequential as we've operated on that before
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
for layer in vgg16_model.layers[:-1]:  #-1 to get rid of last layer
    model.add(layer)
model.summary() 


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
#freezing layers
for layer in model.layers: 
    layer.trainable = False

model.add(Dense(1024, activation = 'relu'))
model.add(Dense(120, activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (224, 224),
                                                 color_mode= 'rgb',
                                                 batch_size = 100)  

Found 10225 images belonging to 120 classes.


In [0]:
# Compiling the CNN
from keras import optimizers
adam = optimizers.Adam(lr = 1e-4)
model.compile(adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit_generator(training_set,
                         samples_per_epoch = 100000,
                         nb_epoch = 20)





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=1000, epochs=20)`
  import sys


Epoch 1/20
1000/1000 [==============================] - 8004s 8s/step - loss: 4.2480 - acc: 0.0814
Epoch 2/20
  17/1000 [..............................] - ETA: 39:40 - loss: 3.8348 - acc: 0.1465

1000/1000 [==============================] - 2480s 2s/step - loss: 3.5984 - acc: 0.1772
Epoch 3/20
  95/1000 [=>............................] - ETA: 37:01 - loss: 3.3933 - acc: 0.2071

1000/1000 [==============================] - 2453s 2s/step - loss: 3.2790 - acc: 0.2275
Epoch 4/20
 131/1000 [==>...........................] - ETA: 34:48 - loss: 3.1540 - acc: 0.2475

1000/1000 [==============================] - 2433s 2s/step - loss: 3.0848 - acc: 0.2619
Epoch 5/20
 148/1000 [===>..........................] - ETA: 34:39 - loss: 3.0040 - acc: 0.2757

...

1000/1000 [==============================] - 2501s 3s/step - loss: 2.5304 - acc: 0.3705
Epoch 11/20
 160/1000 [===>..........................] - ETA: 34:23 - loss: 2.4952 - acc: 0.3782

1000/1000 [==============================] - 2521s 3s/step - loss: 2.4682 - acc: 0.3847
Epoch 12/20
 160/1000 [===>..........................] - ETA: 34:38 - loss: 2.4411 - acc: 0.3877

1000/1000 [==============================] - 2503s 3s/step - loss: 2.4155 - acc: 0.3947
Epoch 13/20
 160/1000 [===>..........................] - ETA: 35:35 - loss: 2.4101 - acc: 0.3952

1000/1000 [==============================] - 2524s 3s/step - loss: 2.3682 - acc: 0.4060
Epoch 14/20
 160/1000 [===>..........................] - ETA: 34:27 - loss: 2.3447 - acc: 0.4100

1000/1000 [==============================] - 2460s 2s/step - loss: 2.3171 - acc: 0.4178
Epoch 15/20
  13/1000 [..............................] - ETA: 40:33 - loss: 2.2775 - acc: 0.4262